### Log into ArcGIS Online

In [15]:
from arcgis.gis import GIS

import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter("ignore", InsecureRequestWarning)

gis = GIS("home")

In [16]:
cm_id = "03209b91e33f4c588007319d18cd6e70"
cm_item = gis.content.get(cm_id)

inventory_table = cm_item.tables[1]
dependencies_table = cm_item.tables[2]

In [17]:
# Function to get the status (authoritative, deprecated, none) of items
def get_status(status):

    if not status:
        return "None"
    else:
        return status.lstrip("org_").title()

In [18]:
# Nope I need to do this now; otherwise it's mixing a string and a null and it'll just get ugly
def get_item_url_str(item):
    if item.type in ["Vector Tile Service", "Feature Service", "Map Service", "Image Service"]:
        return f'<a target="_blank" href="{item.url}">View Service</a>'
    elif item.type == "Web Map":
        return f'<a target="_blank" href="https://www.maps.arcgis.com/apps/mapviewer/index.html?webmap={item.id}">View Map</a>'
    elif item.type in ["Hub Site Application", "StoryMap", "Web Experience", "Web Experience Template", "Dashboard", "Hub Page"]:
        return f'<a target="_blank" href="{item.url}">View App</a>'
    else:
        return "Not applicable"

In [19]:
# Get all users in the org
all_items = gis.content.search(query="*", max_items=10000)

# Initialize empty dict to hold items and dependencies rows for AGOL table
items_list = []
dependencies_list = []

# Loop through users; populate users dictionary
for item in all_items:

    # Initialize two dicts I need for each row to be added to users table
    items_dict = {}
    update_items_dict = {}
    
    # Get the dependencies for the item (list of items)
    dependencies = item.get_dependencies(deep=True)

    # Add k/v pairs to the inner dict
    items_dict["item_id"] = item.id
    items_dict["item_url"] = f'<a target="_blank" href="https://www.maps.arcgis.com/home/item.html?id={item.id}">View Item Page</a>'

    # Could fine-tune this so it returns link text based on the item type...too lazy right now...
    items_dict["service_url"] = get_item_url_str(item)
    items_dict["item_type"] = item.type
    items_dict["title"] = item.title
    items_dict["owner"] = item.owner

    # Well the sharing is going to be a little more complex...will have to work with this a bit; 
    # shared_with returns a dict, "groups" and "level" as keys
    items_dict["sharing"] = str(item.sharing.sharing_level.value).title()

    items_dict["status"] = get_status(item.content_status)

    items_dict["date_created"] = item.created
    items_dict["date_modified"] = item.modified

    # BEGIN DEPENDENCIES
    if not dependencies:
        items_dict["has_dependencies"] = "No"
        items_dict["dependencies_count"] = 0

    else:
        items_dict["has_dependencies"] = "Yes"
        items_dict["dependencies_count"] = len(dependencies)

        for dependency in dependencies:

            dependencies_dict = {}
            update_dependencies_dict = {}

            dependencies_dict["origin_id"] = item.id
            dependencies_dict["origin_title"] = item.title
            dependencies_dict["dependent_id"] = dependency.id
            dependencies_dict["dependent_title"] = dependency.title
            dependencies_dict["dependent_type"] = dependency.type
            dependencies_dict["dependent_item_url"] = f'<a target="_blank" href="https://www.maps.arcgis.com/home/item.html?id={dependency.id}">View Item Page</a>'
            dependencies_dict["dependent_url"] = get_item_url_str(dependency)

            update_dependencies_dict["attributes"] = dependencies_dict

            dependencies_list.append(update_dependencies_dict)

    update_items_dict["attributes"] = items_dict

    items_list.append(update_items_dict)

print(f"Number of items to add to table: {len(items_list)}")
print(f"Number of items to add to table: {len(dependencies_list)}")

Number of items to add to table: 223
Number of items to add to table: 402


### Add dependencies to the dependencies table

In [20]:
# This script will effectively overwrite the output of the last run, 
# so before writing any new records we simply truncate tables
# NOTE: can't use truncate like for users table because of the relationship class!
inventory_table.delete_features(where="1=1")

# Add users to table
items_update = inventory_table.edit_features(adds=items_list)

In [21]:
# This script will effectively overwrite the output of the last run, 
# so before writing any new records we simply truncate tables
# NOTE: can't use truncate like for users table because of the relationship class!
dependencies_table.delete_features(where="1=1")

# Add users to table
dependencies_update = dependencies_table.edit_features(adds=dependencies_list)